<a href="https://colab.research.google.com/github/DenisKai7/Sistem-Rekomendasi-Film/blob/main/Project_Sistem_Rekomendasi_Film.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Dependencies

In [1]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Import Data CSV ke Google  Colab

In [2]:
# load data csv
movies_data = pd.read_csv('/content/movies.csv')

In [3]:
# show 5 baris tabel
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [4]:
# jumlah baris dan kolom di data frame

movies_data.shape

(4803, 24)

In [5]:
# Memilih Fitur yang Sesuai untuk direkomendasikan

selected_features = ['genres','keywords','tagline','cast','director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [6]:
# Mengganti data yang bernilai null dengan null tipe string

for feature in selected_features:
  movies_data[feature] = movies_data[feature].fillna('')

In [7]:
# Menggabung semua fitur rekomendasi yang sesuai
combined_features = movies_data['genres']+' '+movies_data['keywords']+' '+movies_data['tagline']+' '+movies_data['cast']+' '+movies_data['director']

In [8]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [9]:
# konvert data berupa teks ke fungsi vektor

vectorizer = TfidfVectorizer()

In [10]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [11]:
print(feature_vectors)

  (0, 2431)	0.17272411194153
  (0, 7754)	0.1128035714854756
  (0, 13023)	0.1942362060108871
  (0, 10228)	0.16058685400095302
  (0, 8755)	0.22709015857011816
  (0, 14607)	0.15150672398763912
  (0, 16667)	0.19843263965100372
  (0, 14063)	0.20596090415084142
  (0, 13318)	0.2177470539412484
  (0, 17289)	0.20197912553916567
  (0, 17006)	0.23643326319898797
  (0, 13348)	0.15021264094167086
  (0, 11502)	0.27211310056983656
  (0, 11191)	0.09049319826481456
  (0, 16997)	0.1282126322850579
  (0, 15260)	0.07095833561276566
  (0, 4944)	0.24025852494110758
  (0, 14270)	0.21392179219912877
  (0, 3224)	0.24960162956997736
  (0, 16586)	0.12549432354918996
  (0, 14377)	0.33962752210959823
  (0, 5835)	0.1646750903586285
  (0, 3064)	0.22208377802661425
  (0, 3677)	0.21392179219912877
  (0, 5436)	0.1036413987316636
  :	:
  (4801, 17265)	0.2886098184932947
  (4801, 4834)	0.24713765026963996
  (4801, 403)	0.17727585190343226
  (4801, 6934)	0.2886098184932947
  (4801, 11662)	0.21557500762727902
  (4801, 1671

Seleksi Identik dengan Cosine

In [12]:
# Menentukan hasil identik dengan Nilai Identik Cosine

similarity = cosine_similarity(feature_vectors)

In [13]:
print(similarity)

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


In [14]:
print(similarity.shape)

(4803, 4803)


User menginputkan judul film

In [15]:
# Inputkan judul film dari user

movie_name = input(' Enter your favourite movie name : ')

 Enter your favourite movie name : minions


In [16]:
# Membuat list semua judul film di dalam dataset

list_of_all_titles = movies_data['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [17]:
# Mencari dan menemukan hasil terdekat dari judul film yang diinputkan user

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['Minions', 'Admission', 'Domino']


In [18]:
close_match = find_close_match[0]
print(close_match)

Minions


In [19]:
# Mencari dan Menemukan index dari judul film

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
print(index_of_the_movie)

546


In [20]:
# Memunculkan film rekomendasi yang mirip

similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, 0.015080480869558498), (1, 0.026324549609842277), (2, 0.0076906219473718), (3, 0.01107785849067944), (4, 0.00731944021738226), (5, 0.015622197509637828), (6, 0.049650863349780874), (7, 0.007587208629209185), (8, 0.034853197669494164), (9, 0.00741270574434823), (10, 0.008454640520414431), (11, 0.006677506140367064), (12, 0.007675010321856148), (13, 0.006929468112137274), (14, 0.01732446231234333), (15, 0.015523540685383174), (16, 0.007194286593019787), (17, 0.007342038373679808), (18, 0.0296587629603644), (19, 0.013077474681662836), (20, 0.015017890363994268), (21, 0.00630841680645116), (22, 0.013619789990421259), (23, 0.006499809751760643), (24, 0.014225847196536554), (25, 0.0), (26, 0.007830659136887552), (27, 0.006847407398191973), (28, 0.00794231790202792), (29, 0.007864576330432492), (30, 0.022821933704241265), (31, 0.007387092785221709), (32, 0.015042728998812135), (33, 0.0075400670274226), (34, 0.05910439866393935), (35, 0.018362358102754918), (36, 0.0165459500000477), (37, 

In [21]:
len(similarity_score)

4803

In [22]:
# Sorting film berdasarkan dari nilai kemiripan

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
print(sorted_similar_movies)

[(546, 1.0000000000000002), (144, 0.13624591028652155), (3710, 0.12838658420392943), (1113, 0.11609744262644275), (3086, 0.11301513376262114), (3048, 0.111868395933726), (698, 0.10930622947188054), (2823, 0.10921209036986244), (966, 0.10841832624117603), (2481, 0.10413011088728871), (1863, 0.09933685249385445), (2566, 0.09772352766330002), (3334, 0.09711946058938965), (2098, 0.0959968551665252), (1040, 0.09471479916057429), (578, 0.0934341059284286), (848, 0.0928714013999734), (2892, 0.09136353441188094), (1823, 0.09109789810683816), (1124, 0.09060496437328021), (3670, 0.08986052634523536), (1276, 0.0894354030014076), (353, 0.08926335309730457), (131, 0.0890610590411344), (42, 0.08839786622501696), (2351, 0.08779088242115574), (2346, 0.08742383206359698), (2139, 0.08732516178519333), (631, 0.08623776306148367), (2560, 0.0862368810172436), (743, 0.08603393516720147), (719, 0.0846942503996838), (1246, 0.08389827451170843), (359, 0.08353941534965603), (1654, 0.08344029465658086), (2741, 0

In [23]:
# Menampilkan judul film yang mirip berdasarkan dari index

print('Rekomendasi Film Untukmu : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

Rekomendasi Film Untukmu : 

1 . Minions
2 . Mr. Peabody & Sherman
3 . Life During Wartime
4 . 28 Days
5 . Gun Shy
6 . Very Bad Things
7 . The Prince of Egypt
8 . Harold & Kumar Escape from Guantanamo Bay
9 . Marmaduke
10 . How to Deal
11 . Million Dollar Arm
12 . All About Steve
13 . Juno
14 . Tammy
15 . The Flintstones
16 . Alvin and the Chipmunks: The Squeakquel
17 . The Pirates! In an Adventure with Scientists!
18 . Darling Companion
19 . Freddy vs. Jason
20 . The Heat
21 . Running Forever
22 . A Thousand Words
23 . Tropic Thunder
24 . G-Force
25 . Toy Story 3
26 . Love the Coopers
27 . Young Sherlock Holmes
28 . Scary Movie 5
29 . Ice Age


Sistem Rekomendasi Film

In [24]:
movie_name = input(' Masukkan Judul Film Favoritmu : ')

list_of_all_titles = movies_data['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)

print('Rekomendasi Film Untukmu : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

 Masukkan Judul Film Favoritmu : fast furious
Rekomendasi Film Untukmu : 

1 . 2 Fast 2 Furious
2 . Furious 7
3 . Death Race
4 . The Fast and the Furious: Tokyo Drift
5 . Fast Five
6 . The Final Destination
7 . The Fast and the Furious
8 . Days of Thunder
9 . Cars
10 . Gone in Sixty Seconds
11 . Herbie Fully Loaded
12 . Back to the Future Part II
13 . Out of Time
14 . Baby Boy
15 . Four Brothers
16 . Cars 2
17 . American Graffiti
18 . Poetic Justice
19 . Turbo
20 . Speed Racer
21 . Repo Man
22 . Fast Times at Ridgemont High
23 . Here On Earth
24 . Paparazzi
25 . xXx
26 . Pretty Woman
27 . Breakdown
28 . Need for Speed
29 . Driven


In [25]:
#save model
import pickle

# Save vektor dan matrix serupa
with open('vectorizer.sav', 'wb') as f:
  pickle.dump(vectorizer, f)

with open('similarity.sav', 'wb') as f:
  pickle.dump(similarity, f)

In [26]:
#load model
import pickle

# Load Vektor dan matrix
with open('vectorizer.sav', 'rb') as f:
  vectorizer = pickle.load(f)

with open('similarity.sav', 'rb') as f:
  similarity = pickle.load(f)